In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, lower, regexp_extract, split
import os

# spark = SparkSession.builder.appName(
#     "Ch02 - Analyzing the vocabulary of Pride and Prejudice."
# ).setMaster.getOrCreate()

spark = SparkSession.builder.remote("sc://localhost:15002").getOrCreate()

book = spark.read.text("/opt/spark/data/pride-and-prejudice.txt")

lines = book.select(split(col("value"), " ").alias("line"))

words = lines.select(explode(col("line")).alias("word"))

words_lower = words.select(lower(col("word")).alias("word_lower"))
words_clean = words_lower.select(
    regexp_extract(col("word_lower"), "[a-z]*", 0).alias("word")
)
words_nonull = words_clean.where(col("word") != "")

results = words_nonull.groupby(col("word")).count()

results.orderBy(col("count").desc()).show(10)


+----+-----+
|word|count|
+----+-----+
| the| 4306|
|  to| 4139|
|  of| 3593|
| and| 3432|
| her| 2220|
|   a| 1926|
|  in| 1852|
| was| 1839|
|   i| 1754|
| she| 1682|
+----+-----+
only showing top 10 rows



In [39]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf() \
    .setAppName("Apache Iceberg with PySpark") \
    .setAll([
        # ("spark.driver.memory", "1g"),
        # ("spark.executor.memory", "2g"),
        # ("spark.sql.shuffle.partitions", "40"),
        # Add Iceberg SQL extensions like UPDATE or DELETE in Spark
        # ("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"),
        # Register `my_iceberg_catalog`        
        ("spark.sql.catalog.local","org.apache.iceberg.spark.SparkCatalog"),
        ("spark.sql.catalog.local.type","rest"),
        # ("spark.sql.catalog.local.uri", "http://spark-master:8080"),
        ("spark.sql.catalog.local.warehouse","/warehouse")
        # Configure SQL connection to track tables inside `my_iceberg_catalog`
        ("spark.sql.catalog.local.catalog-impl", "org.apache.iceberg.jdbc.JdbcCatalog"),
        ("spark.sql.catalog.local.uri", "jdbc:postgresql://172.17.0.1:6433/iceberg_db"),
        ("spark.sql.catalog.local.jdbc.user", "postgres"),
        ("spark.sql.catalog.local.jdbc.password", "postgres"),
        Configure Warehouse on MinIO
        # ("spark.sql.catalog.my_iceberg_catalog.io-impl", "org.apache.iceberg.aws.s3.S3FileIO"),
        # ("spark.sql.catalog.my_iceberg_catalog.s3.endpoint", "http://minio:9000"),
        # ("spark.sql.catalog.my_iceberg_catalog.s3.path-style-access", "true"),
        # ("spark.sql.catalog.my_iceberg_catalog.warehouse", "s3://warehouse"),
    ])
spark = SparkSession.builder.remote("sc://localhost:15002").config(conf=conf).getOrCreate()

In [40]:
spark.sql("""
      CREATE TABLE IF NOT EXISTS local.db.vaccinations (
        location string,
        date date,
        vaccine string,
        source_url string,
        total_vaccinations bigint,
        people_vaccinated bigint,
        people_fully_vaccinated bigint,
        total_boosters bigint
      ) USING iceberg PARTITIONED BY (location, date)
    """)

SparkConnectGrpcException: (org.apache.iceberg.exceptions.RESTException) Received a success response code of 200, but failed to parse response body into ConfigResponse

In [26]:
from pyspark.sql import SparkSession, DataFrame
import pyspark.sql.functions as F
path = "/opt/spark/data/Belgium.csv"
vaccinations: DataFrame = spark.read \
      .option("header", "true") \
      .option("inferSchema", "true") \
      .csv(path)

vaccinations \
  .withColumn("date", F.to_date(F.col("date"))) \
  .writeTo("local.db.vaccinations") \
  .append()

SparkConnectGrpcException: (org.apache.spark.SparkException) Job aborted due to stage failure: ResultStage 29 (Spark Connect - session_id: "1d45f17e-7d62-462c-8813-1fa4eb04f3c6"
user_context {
  user_id: "qpham"
}
plan {
  command {
    write_operation...) has failed the maximum allowable number of times: 4. Most recent failure reason:
org.apache.spark.shuffle.MetadataFetchFailedException: Missing an output location for shuffle 5 partition 0
	at org.apache.spark.MapOutputTracker$.validateStatus(MapOutputTracker.scala:1739)
	at org.apache.spark.MapOutputTracker$.$anonfun$convertMapStatuses$11(MapOutputTracker.scala:1686)
	at org.apache.spark.MapOutputTracker$.$anonfun$convertMapStatuses$11$adapted(MapOutputTracker.scala:1685)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at org.apache.spark.MapOutputTracker$.convertMapStatuses(MapOutputTracker.scala: